**MODEL V8**

In [1]:
import cv2
import os
import numpy as np
from ultralytics import YOLO
import supervision as sv

GREYHOUND_VIDEO = "../data/raw_videos/20240722NOTG08_V.mp4"

# Initialize the YOLOv8 model
model = YOLO("yolov8n.pt")

# Open the video file
cap = cv2.VideoCapture(GREYHOUND_VIDEO)

# Verify the output directory and permissions
output_dir = "./output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

if not os.access(output_dir, os.W_OK):
    raise PermissionError(f"Write permission denied for the directory {output_dir}")

# Define the output video path
output_path = os.path.join(output_dir, "object_counting_output_v8_3.mp4")

assert cap.isOpened(), "Error reading video file"

# Get video properties
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Initialize VideoWriter with a successful FourCC code
fourcc_code = cv2.VideoWriter_fourcc(*"mp4v")
video_writer = cv2.VideoWriter(output_path, fourcc_code, fps, (w, h))

if not video_writer.isOpened():
    raise IOError(f"Error initializing video writer with path {output_path}")

# Custom function to filter only human detections
def filter_greyhound_detections(detections):
    greyhound_class_id = 16
    mask = detections.class_id == greyhound_class_id
    filtered_detections = sv.Detections(
        xyxy=detections.xyxy[mask],
        confidence=detections.confidence[mask],
        class_id=detections.class_id[mask]
    )
    return filtered_detections

# Process video frames
box_annotator = sv.BoxAnnotator(thickness=4)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    # Perform object detection
    results = model(frame, imgsz=1280)

    # Extract bounding boxes, confidences, and class IDs
    boxes = results[0].boxes.xyxy.cpu().numpy()
    confidences = results[0].boxes.conf.cpu().numpy()
    class_ids = results[0].boxes.cls.cpu().numpy().astype(int)

    # Create detections object
    detections = sv.Detections(
        xyxy=boxes,
        confidence=confidences,
        class_id=class_ids
    )

    greyhound_detections = filter_greyhound_detections(detections)

    # Annotate the frame
    annotated_frame = box_annotator.annotate(scene=frame, detections=greyhound_detections)

    # Write the processed frame to the output video
    video_writer.write(annotated_frame)

cap.release()
video_writer.release()
cv2.destroyAllWindows()

print(f"Processed video saved to {output_path}")



0: 736x1280 5 persons, 2 cars, 1 motorcycle, 2 trucks, 54.8ms
Speed: 6.0ms preprocess, 54.8ms inference, 1.1ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 5 persons, 1 car, 2 trucks, 56.7ms
Speed: 2.4ms preprocess, 56.7ms inference, 0.5ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 5 persons, 1 car, 2 trucks, 50.5ms
Speed: 2.4ms preprocess, 50.5ms inference, 0.5ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 5 persons, 2 cars, 2 trucks, 84.7ms
Speed: 2.3ms preprocess, 84.7ms inference, 0.5ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 6 persons, 2 cars, 2 trucks, 57.3ms
Speed: 2.1ms preprocess, 57.3ms inference, 0.5ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 6 persons, 2 cars, 2 trucks, 49.7ms
Speed: 2.4ms preprocess, 49.7ms inference, 0.8ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 6 persons, 2 cars, 1 motorcycle, 2 trucks, 57.0ms
Speed: 4.1ms preprocess, 57.0ms inference, 